In [1]:
%matplotlib inline
import numpy as np
import json
import seaborn as sns
import pandas as pd
import os
import glob
import SimpleITK as itk
import matplotlib.pyplot as plt
import matplotlib.patheffects as path_effects
sns.set_style("whitegrid")
from src.utils import save_json, subfiles, join, maybe_mkdir_p, reconstruct_seg_df_from_json, reconstruct_calib_df_from_json, reconstruct_UED_df_from_json



# Statistics for different studies

In [21]:
from statsmodels.stats.multitest import multipletests
from statsmodels.stats.multitest import fdrcorrection

groups = ['Baseline','No TTA', 'MC Dropout','Ensemble','Snapshots', "Complex" ,'PhiSeg']
from scipy.stats import wilcoxon
def calcualte_test(x, y, method = wilcoxon):
    # method could also be pearsonr
    nas = np.logical_or(np.isnan(x), np.isnan(y)) # remove nan

    return method(x[~nas], y[~nas])

# if data is screwed
def bootstrap_confidence_interval(data, func=np.median, confidence=95, size=10000):
    """
    func could be np.mean/np.median
    """
    
    bs_replicates = np.empty(size)
    
    # Create bootstrap replicates as much as size
    for i in range(size):
        # Create a bootstrap sample
        bs_sample = np.random.choice(data,size=len(data))
        # Get bootstrap replicate and append to bs_replicates
        bs_replicates[i] = func(bs_sample)
        
    conf_interval = np.percentile(bs_replicates,[100-confidence,confidence])
    np.round(func(bs_replicates), 2), tuple(np.round(conf_interval,2))
    m = np.round(func(bs_replicates), 2)
    ci0, ci1 = np.round(conf_interval,2)
    #return_str = str(m)+'('+str(ci0)+'-'+ str(ci1)+')'
    return_str = f'{m:.2f}({ci0:.2f}-{ci1:.2f})'
    return return_str


def print_tests(data, exp):
    """
    !!!baseline data frist!!!
    """
    print(exp)
    for gtv in ["GTV-T", "GTV-N"]:
        print(gtv)
        pvalues = [] 
        means = []
        for group in groups:
            #print("----------------------")
            #
            test_all_data = data[((data["Ucertainty estimation methods"]==group) | (data["Ucertainty estimation methods"]=="Baseline")) & (data["GTV"]==gtv) ]
            base = test_all_data[(test_all_data["Ucertainty estimation methods"]=="Baseline")][exp].astype({exp: float})
            test = test_all_data[(test_all_data["Ucertainty estimation methods"]==group)][exp].astype({exp: float})
            means.append(str(bootstrap_confidence_interval(test.dropna())))
            if group == "Baseline":
                #pvalues.append(1)
                continue

            s, p = calcualte_test(base.values,test.values)
            # = np.round(p, 5)
            #print(type(p), p)
            pvalues.append(p)
        print("pvalues before correction: ", pvalues)
        corrected_pvalues = fdrcorrection(pvalues)
        #print("pvalues after correction:", corrected_pvalues[0])
        corrected = [1]+ list(corrected_pvalues[1])
        significant = [1]+ list(corrected_pvalues[0])
        print("pvalues after correction:", corrected)

        for i, p in enumerate(corrected):
            #print(significant[i], p)
            if p < 0.001:
                print(means[i]+"***")
            elif p < 0.01:
                print(means[i]+"**")
            elif p < 0.05:
                print(means[i]+"*"+str(np.round(p, 4)))
            else:
                #print(str(np.round(p, 3)))
                p = np.round(p, 4)
                print(means[i]+'    '+str(p))


## Segmentation performance comparision 
Internal

In [18]:
## directorys for internal  probability maps 

DROP2_PMAPS="/data/jintao/nnUNet/nnUNet_results/nnUNet/3d_fullres/Task901_AUH/nnUNetTrainerV2_dropout__nnUNetPlansv2.1/prob_maps/Task901_AUH/imagesTs"
NNUNET_PMAPS="/data/jintao/nnUNet/nnUNet_results/nnUNet/3d_fullres/Task901_AUH/nnUNetTrainerV2__nnUNetPlansv2.1/prob_maps/Task901_AUH/imagesTs"
PH_PMAPS="/data/jintao/nnUNet/nnUNet_results/nnUNet/3d_fullres/Task901_AUH/nnUNetTrainerV2_PhiSeg__nnUNetPlansv2.1/prob_maps/Task901_AUH/imagesTs"
PH_GAMM_PMAP="/data/jintao/nnUNet/nnUNet_results/nnUNet/3d_fullres/Task901_AUH/nnUNetTrainerV2_PhiSeg_gamma__nnUNetPlansv2.1/prob_maps/Task901_AUH/imagesTs"

studies = [
join(NNUNET_PMAPS,'f0_tta'),
join(NNUNET_PMAPS,'f0'),
join(DROP2_PMAPS,'f0_mc10_tta')
,join(NNUNET_PMAPS,'f01234_tta')
,join(DROP2_PMAPS,'f0_tta_snap')
,join(DROP2_PMAPS,'f01234_mc10_tta_snap')
,join(PH_PMAPS,'f01234_tta')
]

groups = ['Baseline','No TTA', 'MC Dropout','Ensemble','Snapshots', "Complex" ,'PhiSeg']
for i, study in enumerate(studies):
    gtvt_seg, gtvn_seg  = reconstruct_seg_df_from_json(study)
    df_seg_t = pd.DataFrame.from_dict(gtvt_seg).T
    df_seg_n = pd.DataFrame.from_dict(gtvn_seg).T

    dft = df_seg_t
    dfn = df_seg_n

    dft['Ucertainty estimation methods'] = groups[i]
    dfn['Ucertainty estimation methods'] = groups[i]

    dft.index.names = ['PatientID']
    dfn.index.names = ['PatientID']
    dft= dft.reset_index(drop=False)
    dfn= dfn.reset_index(drop=False)

    if i == 0:
        dft_all_correct = dft
        dfn_all_correct = dfn
    else:
        dft_all_correct = pd.concat([dft_all_correct, dft])
        dfn_all_correct = pd.concat([dfn_all_correct, dfn])

all_seg_internal = pd.concat([dft_all_correct, dfn_all_correct])
all_seg_internal= all_seg_internal.reset_index(drop=False)


In [19]:
all_seg_internal[all_seg_internal['PatientID']=='HNCDL_280']

,index,PatientID,Accuracy,Mean Surface Distance (mm),DSC,False Discovery Rate,False Negative Rate,False Omission Rate,False Positive Rate,HD95 (mm),...,Precision,Recall,Surface Dice 1mm,Surface Dice 2mm,Surface Dice 3mm,Volume (cc),Pred Volume (cc),True Negative Rate,GTV,Ucertainty estimation methods
47,47,HNCDL_280,0.998736,1.737603,0.836463,0.216239,0.103238,0.000374,0.000895,5.830952,...,0.783761,0.896762,0.466051,0.670594,0.83136,68.88,78.81,0.999105,GTV-T,Baseline
144,47,HNCDL_280,0.998704,1.789995,0.830953,0.215644,0.116564,0.000422,0.000879,6.63325,...,0.784356,0.883436,0.465602,0.665067,0.816441,68.89,77.59,0.999121,GTV-T,No TTA
241,47,HNCDL_280,0.998809,1.65454,0.844814,0.202821,0.101497,0.000368,0.000827,5.0,...,0.797179,0.898503,0.486957,0.686233,0.842676,68.89,77.64,0.999173,GTV-T,MC Dropout
338,47,HNCDL_280,0.998815,1.693324,0.849298,0.215827,0.07378,0.000267,0.000923,5.0,...,0.784173,0.92622,0.495324,0.694416,0.849226,68.88,81.36,0.999077,GTV-T,Ensemble
435,47,HNCDL_280,0.998875,1.540209,0.8514,0.187147,0.106214,0.000385,0.000745,5.0,...,0.812853,0.893786,0.507717,0.712851,0.867791,68.89,75.75,0.999255,GTV-T,Snapshots
532,47,HNCDL_280,0.998861,1.574832,0.850194,0.191527,0.103543,0.000375,0.000769,5.09902,...,0.808473,0.896457,0.506782,0.708568,0.863735,68.89,76.39,0.999231,GTV-T,Complex
629,47,HNCDL_280,0.998768,1.691543,0.837751,0.202483,0.11774,0.000426,0.000811,6.0,...,0.797517,0.88226,0.478164,0.681341,0.835272,68.89,76.21,0.999189,GTV-T,PhiSeg
726,47,HNCDL_280,0.999778,11.768197,0.019866,0.981577,0.978446,0.000102,0.00012,22.219359,...,0.018423,0.021554,0.067598,0.131107,0.216344,2.0,2.33,0.99988,GTV-N,Baseline
823,47,HNCDL_280,0.999791,26.28007,0.039952,0.962204,0.957631,0.000098,0.000111,65.949223,...,0.037796,0.042369,0.040377,0.083435,0.154697,1.96,2.2,0.999889,GTV-N,No TTA
920,47,HNCDL_280,0.99981,46.281824,0.0,1.0,1.0,0.000103,0.000088,69.116538,...,0.0,0.0,0.0,0.0,0.0,1.96,1.67,0.999912,GTV-N,MC Dropout


In [22]:
exp= 'DSC' 
#exp= 'HD95 (mm)'
#exp= 'Mean Surface Distance (mm)'
#exp= 'Surface Dice 2mm'

print_tests(all_seg_internal, exp)

DSC
GTV-T
pvalues before correction:  [0.008971217549269767, 0.13180683322738812, 0.002308022296133644, 0.04035775996940854, 0.4515773027033496, 0.00772192657235769]
pvalues after correction: [1, 0.017942435098539534, 0.15816819987286573, 0.013848133776801864, 0.06053663995411281, 0.4515773027033496, 0.017942435098539534]
0.75(0.72-0.80)    1
0.74(0.72-0.78)*0.0179
0.74(0.70-0.78)    0.1582
0.76(0.72-0.80)*0.0138
0.74(0.69-0.77)    0.0605
0.76(0.72-0.77)    0.4516
0.73(0.69-0.77)*0.0179
GTV-N
pvalues before correction:  [0.011697936760166063, 0.10313682673243209, 0.005390649509551264, 0.3744737555342701, 0.21150382409725055, 0.005199335321500139]
pvalues after correction: [1, 0.023395873520332126, 0.15470524009864814, 0.016171948528653793, 0.3744737555342701, 0.25380458891670066, 0.016171948528653793]
0.79(0.76-0.82)    1
0.78(0.76-0.81)*0.0234
0.78(0.76-0.82)    0.1547
0.79(0.78-0.82)*0.0162
0.78(0.74-0.82)    0.3745
0.80(0.76-0.82)    0.2538
0.78(0.74-0.81)*0.0162


External

In [8]:
## directorys for external probability maps 

DROP2_PMAPS="/data/jintao/nnUNet/nnUNet_results/nnUNet/3d_fullres/Task901_AUH/nnUNetTrainerV2_dropout__nnUNetPlansv2.1/prob_maps/Task711_NKI_origin/imagesAll_ct_correct"
NNUNET_PMAPS="/data/jintao/nnUNet/nnUNet_results/nnUNet/3d_fullres/Task901_AUH/nnUNetTrainerV2__nnUNetPlansv2.1/prob_maps/Task711_NKI_origin/imagesAll_ct_correct"
PH_PMAPS="/data/jintao/nnUNet/nnUNet_results/nnUNet/3d_fullres/Task901_AUH/nnUNetTrainerV2_PhiSeg__nnUNetPlansv2.1/prob_maps/Task711_NKI_origin/imagesAll_ct_correct"
PH_GAMM_PMAP="/data/jintao/nnUNet/nnUNet_results/nnUNet/3d_fullres/Task901_AUH/nnUNetTrainerV2_PhiSeg_gamma__nnUNetPlansv2.1/prob_maps/Task711_NKI_origin/imagesAll_ct_correct"

studies = [
join(NNUNET_PMAPS,'f0_tta'),
join(NNUNET_PMAPS,'f0'),
join(DROP2_PMAPS,'f0_mc10_tta')
,join(NNUNET_PMAPS,'f01234_tta')
,join(DROP2_PMAPS,'f0_tta_snap')
,join(DROP2_PMAPS,'f01234_mc10_tta_snap')
,join(PH_PMAPS,'f01234_tta')
]
#,join(PH_GAMM_PMAP,'f0_tta')]


groups = ['Baseline', 'No TTA','MC Dropout','Ensemble','Snapshots', "Complex" ,'PhiSeg']
for i, study in enumerate(studies):
    gtvt_seg, gtvn_seg  = reconstruct_seg_df_from_json(study)
    df_seg_t = pd.DataFrame.from_dict(gtvt_seg).T
    df_seg_n = pd.DataFrame.from_dict(gtvn_seg).T

    dft = df_seg_t
    dfn = df_seg_n

    dft['Ucertainty estimation methods'] = groups[i]
    dfn['Ucertainty estimation methods'] = groups[i]

    dft.index.names = ['PatientID']
    dfn.index.names = ['PatientID']
    dft= dft.reset_index(drop=False)
    dfn= dfn.reset_index(drop=False)

    if i == 0:
        dft_all_correct = dft
        dfn_all_correct = dfn
    else:
        dft_all_correct = pd.concat([dft_all_correct, dft])
        dfn_all_correct = pd.concat([dfn_all_correct, dfn])


all_seg_external = pd.concat([dft_all_correct, dfn_all_correct])
all_seg_external= all_seg_external.reset_index(drop=False)

# exp= 'DSC' 
# exp= 'HD95 (mm)'
# exp= 'Mean Surface Distance (mm)'
exp= 'Surface Dice 2mm'
print_tests(all_seg_external, exp)

Surface Dice 2mm
GTV-T
pvalues before correction:  [0.049309693183975635, 0.10734313335396876, 2.0498904903972427e-08, 0.0004754376320679146, 1.0957672261788388e-07, 0.3491034180382393]
pvalues after correction: [1, 0.14075463587469092, 0.2031637184296896, 1.229934231207582e-07, 0.0019003947124421267, 5.478834930188512e-07, 0.3491034180382393]
0.57(0.55-0.60)    1
0.56(0.54-0.59)    0.1408
0.56(0.53-0.59)    0.2032
0.59(0.57-0.61)***
0.58(0.54-0.61)**
0.61(0.56-0.62)***
0.56(0.53-0.60)    0.3491
GTV-N
pvalues before correction:  [0.004563590323981993, 0.16680243408877604, 3.0479477750938654e-07, 0.49053175526994763, 0.12838190966958027, 3.11428497349035e-11]
pvalues after correction: [1, 0.018129782894158668, 0.337815964505617, 1.523972958548652e-06, 0.49053175526994763, 0.337815964505617, 1.8685709839487287e-10]
0.67(0.65-0.71)    1
0.66(0.65-0.69)*
0.66(0.62-0.72)    0.3378
0.72(0.66-0.76)***
0.67(0.64-0.72)    0.4905
0.70(0.67-0.73)    0.3378
0.64(0.60-0.69)***


## Calibarion performance comparision 
Internal

In [9]:

groups = ['Baseline','No TTA', 'MC Dropout','Ensemble','Snapshots',"Complex", 'PhiSeg' ]
## directorys for main internal probability maps 

DROP2_PMAPS="/data/jintao/nnUNet/nnUNet_results/nnUNet/3d_fullres/Task901_AUH/nnUNetTrainerV2_dropout__nnUNetPlansv2.1/prob_maps/Task901_AUH/imagesTs"
NNUNET_PMAPS="/data/jintao/nnUNet/nnUNet_results/nnUNet/3d_fullres/Task901_AUH/nnUNetTrainerV2__nnUNetPlansv2.1/prob_maps/Task901_AUH/imagesTs"
PH_PMAP="/data/jintao/nnUNet/nnUNet_results/nnUNet/3d_fullres/Task901_AUH/nnUNetTrainerV2_PhiSeg__nnUNetPlansv2.1/prob_maps/Task901_AUH/imagesTs"
PH_GAMM_PMAP="/data/jintao/nnUNet/nnUNet_results/nnUNet/3d_fullres/Task901_AUH/nnUNetTrainerV2_PhiSeg_gamma__nnUNetPlansv2.1/prob_maps/Task901_AUH/imagesTs"

studies = [
join(NNUNET_PMAPS,'f0_tta'),
join(NNUNET_PMAPS,'f0'),
join(DROP2_PMAPS,'f0_mc10_tta')
,join(NNUNET_PMAPS,'f01234_tta')
#,join(NNUNET_PMAPS,'f012456_tta')
,join(DROP2_PMAPS,'f0_tta_snap')
,join(DROP2_PMAPS,'f01234_mc10_tta_snap')
,join(PH_PMAP,'f01234_tta')

]
#,join(PH_GAMM_PMAP,'f0_tta')]

for i, study in enumerate(studies):
    gtvt_calib, gtvn_calib  = reconstruct_calib_df_from_json(study)
    df_cal_t = pd.DataFrame.from_dict(gtvt_calib).T
    df_cal_n = pd.DataFrame.from_dict(gtvn_calib).T

    df_cal_t['GTV'] = 'GTV-T'
    
    df_cal_n['GTV'] = 'GTV-N'

    dft = df_cal_t
    dfn = df_cal_n
    
    dft['Ucertainty estimation methods'] = groups[i]
    dfn['Ucertainty estimation methods'] = groups[i]
    dft.index.names = ['PatientID']
    dfn.index.names = ['PatientID']
    dft= dft.reset_index(drop=False)
    dfn= dfn.reset_index(drop=False)

    if i == 0:
        dft_all = dft
        dfn_all = dfn
    else:
        dft_all = pd.concat([dft_all, dft])
        dfn_all = pd.concat([dfn_all, dfn])

all_calib_internal = pd.concat([dft_all, dfn_all])
all_calib_internal= all_calib_internal.reset_index(drop=True)

## directorys for external probability maps 

DROP2_PMAPS_EXT="/data/jintao/nnUNet/nnUNet_results/nnUNet/3d_fullres/Task901_AUH/nnUNetTrainerV2_dropout__nnUNetPlansv2.1/prob_maps/Task711_NKI_origin/imagesAll_ct_correct"
NNUNET_PMAPS_EXT="/data/jintao/nnUNet/nnUNet_results/nnUNet/3d_fullres/Task901_AUH/nnUNetTrainerV2__nnUNetPlansv2.1/prob_maps/Task711_NKI_origin/imagesAll_ct_correct"
PH_PMAPS_EXT="/data/jintao/nnUNet/nnUNet_results/nnUNet/3d_fullres/Task901_AUH/nnUNetTrainerV2_PhiSeg__nnUNetPlansv2.1/prob_maps/Task711_NKI_origin/imagesAll_ct_correct"
PH_GAMM_PMAPS_EXT="/data/jintao/nnUNet/nnUNet_results/nnUNet/3d_fullres/Task901_AUH/nnUNetTrainerV2_PhiSeg_gamma__nnUNetPlansv2.1/prob_maps/Task711_NKI_origin/imagesAll_ct_correct"

studies = [
join(NNUNET_PMAPS_EXT,'f0_tta'),#X8
join(NNUNET_PMAPS_EXT,'f0'), #X1
join(DROP2_PMAPS_EXT,'f0_mc10_tta') #X80
,join(NNUNET_PMAPS_EXT,'f01234_tta') #X40
#,join(NNUNET_PMAPS_EXT,'f012456_tta')
,join(DROP2_PMAPS_EXT,'f0_tta_snap') #X40
,join(DROP2_PMAPS_EXT,'f01234_mc10_tta_snap') #X2000
,join(PH_PMAPS_EXT,'f01234_tta') #X40
]

for i, study in enumerate(studies):
    gtvt_calib, gtvn_calib  = reconstruct_calib_df_from_json(study)
    df_cal_t = pd.DataFrame.from_dict(gtvt_calib).T
    df_cal_n = pd.DataFrame.from_dict(gtvn_calib).T

    df_cal_t['GTV'] = 'GTV-T'
    df_cal_n['GTV'] = 'GTV-N'
    
    dft = df_cal_t
    dfn = df_cal_n

    dft['Ucertainty estimation methods'] = groups[i]
    dfn['Ucertainty estimation methods'] = groups[i]
    dft.index.names = ['PatientID']
    dfn.index.names = ['PatientID']
    dft= dft.reset_index(drop=False)
    dfn= dfn.reset_index(drop=False)

    if i == 0:
        dft_all = dft
        dfn_all = dfn
    else:
        dft_all = pd.concat([dft_all, dft])
        dfn_all = pd.concat([dfn_all, dfn])
        dft_all= dft_all.reset_index(drop=True)
        dfn_all= dfn_all.reset_index(drop=True)


all_calib_external = pd.concat([dft_all, dfn_all])
all_calib_external= all_calib_external.reset_index(drop=True)


In [11]:
all_calib_internal[all_calib_internal['PatientID']=='HNCDL_280']

,PatientID,Brier Score,Expected Calibration Error,GTV,Ucertainty estimation methods
47,HNCDL_280,0.203773,0.174332,GTV-T,Baseline
144,HNCDL_280,0.231819,0.215019,GTV-T,No TTA
241,HNCDL_280,0.193813,0.161342,GTV-T,MC Dropout
338,HNCDL_280,0.173809,0.127346,GTV-T,Ensemble
435,HNCDL_280,0.180271,0.142972,GTV-T,Snapshots
532,HNCDL_280,0.178794,0.139138,GTV-T,Complex
629,HNCDL_280,0.166637,0.106083,GTV-T,PhiSeg
726,HNCDL_280,0.415561,0.365748,GTV-N,Baseline
823,HNCDL_280,0.638483,0.591941,GTV-N,No TTA
920,HNCDL_280,0.464064,0.399189,GTV-N,MC Dropout


internal

In [13]:
exp= 'Brier Score' 
#exp= 'Expected Calibration Error'

print_tests(all_calib_internal, exp)

Brier Score
GTV-T
pvalues before correction:  [1.941689114941934e-17, 0.01583849928222723, 7.824702812393471e-15, 3.796117156797039e-07, 2.82891648034575e-12, 4.9474681596101975e-09]
pvalues after correction: [1, 1.1650134689651606e-16, 0.01583849928222723, 2.3474108437180414e-14, 4.5553405881564467e-07, 5.6578329606915e-12, 7.421202239415297e-09]
0.27(0.25-0.30)    1
0.32(0.29-0.37)***
0.28(0.24-0.29)*
0.24(0.23-0.27)***
0.25(0.23-0.27)***
0.22(0.20-0.25)***
0.21(0.19-0.23)***
GTV-N
pvalues before correction:  [3.173239043057602e-11, 0.36290398382399647, 2.2764090083752265e-10, 0.00014701249016736017, 3.418736169555222e-11, 1.4802061271264318e-10]
pvalues after correction: [1, 1.0256208508665667e-10, 0.36290398382399647, 3.41461351256284e-10, 0.0001764149882008322, 1.0256208508665667e-10, 2.9604122542528635e-10]
0.24(0.22-0.26)    1
0.27(0.26-0.29)***
0.24(0.21-0.25)    0.3629
0.23(0.21-0.25)***
0.22(0.20-0.24)***
0.20(0.20-0.23)***
0.19(0.17-0.21)***


external

In [20]:
exp= 'Brier Score' 
#exp= 'Expected Calibration Error'

print_tests(all_calib_external, exp)

Brier Score
GTV-T
pvalues before correction:  [5.83741598906476e-34, 5.667112058180076e-08, 2.8937843008242956e-26, 3.0298572041473517e-21, 6.635046019989359e-34, 9.385297574034208e-27]
pvalues after correction: [1, 1.990513805996808e-33, 5.667112058180076e-08, 4.340676451236444e-26, 3.6358286449768216e-21, 1.990513805996808e-33, 1.8770595148068417e-26]
0.36(0.34-0.37)    1
0.43(0.41-0.45)***
0.34(0.32-0.35)***
0.31(0.28-0.33)***
0.31(0.29-0.32)***
0.26(0.25-0.28)***
0.24(0.22-0.26)***
GTV-N
pvalues before correction:  [1.0503229785595869e-29, 0.004264211328385304, 4.238262099832351e-19, 1.39971976501867e-15, 3.113593257175039e-24, 5.618951154620924e-11]
pvalues after correction: [1, 6.301937871357522e-29, 0.004264211328385304, 8.476524199664702e-19, 2.099579647528005e-15, 9.340779771525117e-24, 6.742741385545108e-11]
0.32(0.30-0.35)    1
0.37(0.35-0.42)***
0.30(0.28-0.34)**
0.28(0.25-0.31)***
0.27(0.25-0.30)***
0.26(0.23-0.28)***
0.27(0.24-0.30)***


## Uncertainty Error Overlap Degree (UED) comparision
Internal

In [15]:
## directorys for main internal probability maps 

NNUNET_UREGIONS="/data/jintao/nnUNet/nnUNet_results/nnUNet/3d_fullres/Task901_AUH/nnUNetTrainerV2__nnUNetPlansv2.1/u_regions/Task901_AUH/imagesTs"
DROP1_UREGIONS="/data/jintao/nnUNet/nnUNet_results/nnUNet/3d_fullres/Task901_AUH/nnUNetTrainerV2_dropout1__nnUNetPlansv2.1/u_regions/Task901_AUH/imagesTs"
DROP2_UREGIONS="/data/jintao/nnUNet/nnUNet_results/nnUNet/3d_fullres/Task901_AUH/nnUNetTrainerV2_dropout__nnUNetPlansv2.1/u_regions/Task901_AUH/imagesTs"
DROP3_UREGIONS="/data/jintao/nnUNet/nnUNet_results/nnUNet/3d_fullres/Task901_AUH/nnUNetTrainerV2_dropout3__nnUNetPlansv2.1/u_regions/Task901_AUH/imagesTs"
DROP5_UREGIONS="/data/jintao/nnUNet/nnUNet_results/nnUNet/3d_fullres/Task901_AUH/nnUNetTrainerV2_dropout5__nnUNetPlansv2.1/u_regions/Task901_AUH/imagesTs"

PHISEG_UREGIONS = "/data/jintao/nnUNet/nnUNet_results/nnUNet/3d_fullres/Task901_AUH/nnUNetTrainerV2_PhiSeg__nnUNetPlansv2.1/u_regions/Task901_AUH/imagesTs"
PHISEG_GAMMA_UREGIONS= "/data/jintao/nnUNet/nnUNet_results/nnUNet/3d_fullres/Task901_AUH/nnUNetTrainerV2_PhiSeg_gamma__nnUNetPlansv2.1/u_regions/Task901_AUH/imagesTs"

studies = [
join(NNUNET_UREGIONS,'f0_tta'),
join(NNUNET_UREGIONS,'f0'),
join(DROP2_UREGIONS,'f0_mc10_tta')
,join(NNUNET_UREGIONS,'f01234_tta')
#,join(NNUNET_PMAPS,'f012456_tta')
,join(DROP2_UREGIONS,'f0_tta_snap')
,join(DROP2_UREGIONS,'f01234_mc10_tta_snap')
,join(PHISEG_UREGIONS,'f01234_tta')

]


def compute_best_ued(studies):

    # Define the base directory
    for index, group in enumerate(groups):

        base_dir = studies[index]

        # Define the subdirectories to iterate through
        #subdirs = ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9"] #best th leads to results without 
        subdirs = ["7"]
        # Initialize a dictionary to store the results for each patient

        results = [] 
        # Iterate through the subdirectories
        for gtv in ['GTV-T', 'GTV-N']:
            patient_results = {}
            for subdir in subdirs:
                # Obtain the UED summary.json file path
                ued_path = os.path.join(base_dir, subdir, gtv+"_union", "summary.json")
                # Obtain the FND and FPD summary.json file path
                
                fpnd_path = os.path.join(base_dir, subdir, gtv, "summary.json")
                # Check if the summary.json files exist
                if os.path.exists(ued_path) and os.path.exists(fpnd_path):
                    # Read the UED summary.json file
                    with open(ued_path) as f:
                        ued_data = json.load(f)
                    # Read the FND and FPD summary.json file
                    with open(fpnd_path) as f:
                        fpnd_data = json.load(f)

                    # Iterate through the patients in the UED summary.json file
                    for patient_data in ued_data["results"]["all"]:
                        # Obtain the unique patient ID
                        patient_id = os.path.basename(patient_data["reference"]).split(".nii.gz")[0]
                        # Check if the patient ID already exists in the patient_results dictionary
                        if patient_id not in patient_results:
                            # Create a new dictionary for the patient ID
                            patient_results[patient_id] = {}
                        # Obtain the UED value for the patient
                        ued_value = patient_data["1"]["Dice"]
                        ued_volume = patient_data["1"]["Total Positives Test"]

                        # Check if the UED value is higher than the current value for the patient
                        if "UED" not in patient_results[patient_id]:
                            # Add the UED value to the patient's dictionary
                            patient_results[patient_id]["UED"]= ued_value
                            patient_results[patient_id]["UED_volume"]= ued_volume

                            patient_results[patient_id]["UED_th"]= subdir
                        if ued_value > patient_results[patient_id]["UED"]:
                            # Add the UED value to the patient's dictionary
                            patient_results[patient_id]["UED"] = ued_value
                            patient_results[patient_id]["UED_volume"]= ued_volume
                            patient_results[patient_id]["UED_th"] = subdir
                        if ued_volume == 0:
                            patient_results[patient_id]["UED"]= np.nan
                            patient_results[patient_id]["UED_volume"]= ued_volume
                            patient_results[patient_id]["UED_th"]= 0

                    for patient_data in fpnd_data["results"]["all"]:
                        # Obtain the unique patient ID
                        patient_id = os.path.basename(patient_data["reference"]).split(".nii.gz")[0]
                        if patient_id not in patient_results:
                            print("fError, some thing went wrong {patient_id}")
                            # Create a new dictionary for the patient ID
                            break
                        # Obtain the FND value for the patient
                        fnd_value = patient_data["1"]["Dice"]
                        fnd_volume = patient_data["1"]["Total Positives Test"]
                        # Obtain the FPD value for the patient
                        fpd_value = patient_data["2"]["Dice"]
                        fpd_volume = patient_data["2"]["Total Positives Test"]

                        # Check if the FND value is higher than the current value for the patient
                        if "FND" not in patient_results[patient_id]:
                            # Add the FND value to the patient's dictionary
                            patient_results[patient_id]["FND"] = fnd_value
                            patient_results[patient_id]["FND_volume"]= fnd_volume
                            patient_results[patient_id]["FND_th"] = subdir
                        if fnd_value > patient_results[patient_id]["FND"]:
                            # Add the FND value to the patient's dictionary
                            patient_results[patient_id]["FND"]= fnd_value
                            patient_results[patient_id]["FND_volume"]= fnd_volume
                            patient_results[patient_id]["FND_th"] = subdir
                        if fnd_volume == 0:
                            patient_results[patient_id]["FND"]= np.nan
                            patient_results[patient_id]["FND_volume"]= fnd_volume
                            patient_results[patient_id]["FND_th"]= 0

                        # Check if the FPD value is higher than the current value for the patient
                        if "FPD" not in patient_results[patient_id]:
                            # Add the FPD value to the patient's dictionary
                            patient_results[patient_id]["FPD"] = fpd_value
                            patient_results[patient_id]["FPD_volume"]= fpd_volume
                            patient_results[patient_id]["FPD_th"]= subdir
                        if fpd_value > patient_results[patient_id]["FPD"]:
                            # Add the FPD value to the patient's dictionary
                            patient_results[patient_id]["FPD"] = fpd_value
                            patient_results[patient_id]["FPD_volume"]= fpd_volume
                            patient_results[patient_id]["FPD_th"]= subdir
                        if fpd_volume == 0:
                            patient_results[patient_id]["FPD"]= np.nan
                            patient_results[patient_id]["FPD_volume"]= fpd_volume
                            patient_results[patient_id]["FPD_th"]= 0

                        patient_results[patient_id]["GTV"] = gtv
                        patient_results[patient_id]["Ucertainty estimation methods"] = group
            results.append(patient_results)
        dft = pd.DataFrame(results[0]).T
        dfn = pd.DataFrame(results[1]).T
        dft.index.names = ['PatientID']
        dfn.index.names = ['PatientID']
        dft= dft.reset_index(drop=False)
        dfn= dfn.reset_index(drop=False)

        df_ued = pd.concat([dft, dfn])
        df_ued= df_ued.reset_index(drop=True)

        if index == 0:
            df_ued_all = df_ued
        else:
            df_ued_all = pd.concat([df_ued_all, df_ued], axis=0)
            df_ued_all= df_ued_all.reset_index(drop=True)

    return df_ued_all
    
df_ued_all_internal = compute_best_ued(studies)

DROP2_UREGIONS_EXT="/data/jintao/nnUNet/nnUNet_results/nnUNet/3d_fullres/Task901_AUH/nnUNetTrainerV2_dropout__nnUNetPlansv2.1/u_regions/Task711_NKI_origin/imagesAll_ct_correct"
NNUNET_UREGIONS_EXT="/data/jintao/nnUNet/nnUNet_results/nnUNet/3d_fullres/Task901_AUH/nnUNetTrainerV2__nnUNetPlansv2.1/u_regions/Task711_NKI_origin/imagesAll_ct_correct"
PHISEG_UREGIONS_EXT="/data/jintao/nnUNet/nnUNet_results/nnUNet/3d_fullres/Task901_AUH/nnUNetTrainerV2_PhiSeg__nnUNetPlansv2.1/u_regions/Task711_NKI_origin/imagesAll_ct_correct"

studies_ext = [
join(NNUNET_UREGIONS_EXT,'f0_tta'),
join(NNUNET_UREGIONS_EXT,'f0'),
join(DROP2_UREGIONS_EXT,'f0_mc10_tta')
,join(NNUNET_UREGIONS_EXT,'f01234_tta')
#,join(NNUNET_PMAPS,'f012456_tta')
,join(DROP2_UREGIONS_EXT,'f0_tta_snap')
,join(DROP2_UREGIONS_EXT,'f01234_mc10_tta_snap')
,join(PHISEG_UREGIONS_EXT,'f01234_tta')
]
df_ued_all_external = compute_best_ued(studies_ext)
df_ued_all_external

,PatientID,UED,UED_volume,UED_th,FND,FND_volume,FND_th,FPD,FPD_volume,FPD_th,GTV,Ucertainty estimation methods
0,NKI_002,0.139333,11878,7,0.06278,11085,7,0.155526,793,7,GTV-T,Baseline
1,NKI_003,0.270218,5972,7,0.23644,4295,7,0.121432,1677,7,GTV-T,Baseline
2,NKI_005,0.407866,6621,7,0.32537,4966,7,0.11124,1655,7,GTV-T,Baseline
3,NKI_007,0.323929,5541,7,0.21167,3951,7,0.206078,1590,7,GTV-T,Baseline
4,NKI_010,NaN,0,0,NaN,0,0,NaN,0,0,GTV-T,Baseline
...,...,...,...,...,...,...,...,...,...,...,...,...
3299,NKI_333,0.369438,12535,7,0.338453,9150,7,0.436776,3385,7,GTV-N,PhiSeg
3300,NKI_334,0.0,5098,7,0.0,5098,7,NaN,0,0,GTV-N,PhiSeg
3301,NKI_336,0.309284,6902,7,0.315542,5064,7,0.259931,1838,7,GTV-N,PhiSeg
3302,NKI_338,0.388861,26730,7,0.270884,19457,7,0.602775,7273,7,GTV-N,PhiSeg


In [16]:
df_ued_all_internal[df_ued_all_internal['PatientID']=='HNCDL_280']

,PatientID,UED,UED_volume,UED_th,FND,FND_volume,FND_th,FPD,FPD_volume,FPD_th,GTV,Ucertainty estimation methods
47,HNCDL_280,0.273547,17005,7,0.274642,13112,7,0.272488,3893,7,GTV-T,Baseline
144,HNCDL_280,0.509226,4657,7,0.430792,3541,7,0.634689,1116,7,GTV-N,Baseline
241,HNCDL_280,0.188327,9986,7,0.191048,7631,7,0.186095,2355,7,GTV-T,No TTA
338,HNCDL_280,0.190184,1879,7,0.12909,1455,7,0.270398,424,7,GTV-N,No TTA
435,HNCDL_280,0.286679,17249,7,0.283075,13144,7,0.290334,4105,7,GTV-T,MC Dropout
532,HNCDL_280,0.349899,3290,7,0.209313,2551,7,0.612769,739,7,GTV-N,MC Dropout
629,HNCDL_280,0.300597,22904,7,0.259269,17358,7,0.340752,5546,7,GTV-T,Ensemble
726,HNCDL_280,0.431731,5252,7,0.28286,4348,7,0.898162,904,7,GTV-N,Ensemble
823,HNCDL_280,0.335532,18980,7,0.320308,13950,7,0.35239,5030,7,GTV-T,Snapshots
920,HNCDL_280,0.333579,3871,7,0.256318,3090,7,0.455873,781,7,GTV-N,Snapshots


internal

In [26]:

exp= 'UED'
#exp= 'FPD'
#exp= 'FND'


print_tests(df_ued_all_internal, exp)

UED
GTV-T
pvalues before correction:  [6.596934068523362e-15, 0.9756005131843929, 8.831016239603322e-09, 0.002264892412223735, 7.162651471027388e-06, 8.82522951438495e-11]
pvalues after correction: [1, 3.958160441113952e-14, 0.9756005131843929, 3.53240644904922e-08, 0.004524655086808521, 2.1487800502721347e-05, 4.4126147564136285e-10]
0.28(0.26-0.29)    1
0.21(0.19-0.21)***
0.27(0.26-0.29)    0.9756
0.31(0.29-0.32)***
0.29(0.28-0.30)**
0.31(0.29-0.33)***
0.38(0.35-0.39)***
GTV-N
pvalues before correction:  [1.1773776581830565e-11, 0.5956173388863042, 0.0001319485191368057, 0.29052655909260117, 0.0002484151047972786, 7.144216718797062e-06]
pvalues after correction: [1, 7.064265948890406e-11, 0.5956173388863042, 0.0005276896232658176, 0.49664743664701566, 0.0007450601995286734, 3.572057319930645e-05]
0.28(0.26-0.29)    1
0.22(0.20-0.23)***
0.28(0.26-0.29)    0.5956
0.31(0.29-0.33)***
0.27(0.26-0.29)    0.4966
0.31(0.28-0.33)***
0.36(0.35-0.37)***


external

In [18]:
# exp= 'UED'
# exp= 'FND'
exp= 'FPD'

print_tests(df_ued_all_external, exp)

FPD
GTV-T
pvalues before correction:  [5.7119815668003955e-11, 1.4922206363698853e-33, 3.3244251693779676e-36, 3.302977159797594e-38, 4.3152717533031645e-37, 4.3771866414248054e-33]
pvalues after correction: [1, 5.7119815668003955e-11, 4.476661909109656e-33, 1.329770067751187e-35, 1.981786295878556e-37, 2.1576358766515823e-36, 8.754373282849611e-33]
0.16(0.15-0.17)    1
0.23(0.21-0.24)***
0.34(0.32-0.36)***
0.32(0.30-0.34)***
0.36(0.34-0.38)***
0.37(0.35-0.38)***
0.37(0.35-0.40)***
GTV-N
pvalues before correction:  [2.46237267172169e-07, 2.772638228005272e-26, 1.6277053974326143e-31, 1.5922660989272084e-31, 3.536497583414815e-30, 2.213711843389758e-28]
pvalues after correction: [1, 2.46237267172169e-07, 5.545276456010544e-26, 9.55359659356325e-31, 9.55359659356325e-31, 1.414599033365926e-29, 6.641135530169274e-28]
0.18(0.17-0.19)    1
0.23(0.21-0.25)***
0.34(0.32-0.36)***
0.35(0.33-0.37)***
0.38(0.36-0.40)***
0.37(0.35-0.39)***
0.44(0.41-0.46)***


## Correlation statistics

In [2]:
import SimpleITK as sitk
import numpy as np
from multiprocessing import Pool
from medpy.metric.binary import hd, asd
from scipy.ndimage import binary_erosion
from src.utils import save_json, subfiles, join, maybe_mkdir_p, reconstruct_seg_df_from_json, reconstruct_calib_df_from_json, reconstruct_UED_df_from_json

def calcualte_target_entropy(arguments, only_seg_roi=False):
    th = 0.7
    umap_folder , seg_folder,  filename = arguments

    # create an dict for data storage

    seg_path = join(seg_folder, filename) 
    umap_path = join(umap_folder, filename.replace(".nii.gz", ".npz")) 
    pid = filename.replace(".nii.gz", "")
    
    
    seg = sitk.GetArrayFromImage(sitk.ReadImage(seg_path))
    umap = np.load(umap_path)['umap']

    resutl_list = []

    for target in range(1,3):

        entropy_results = {}
        entropy_results['PatientID'] = pid

        if target == 1:
            gtv = 'GTV-T'
            
        elif target == 2:
            gtv = 'GTV-N'

        # create an dict for each GTV
        entropy_results['GTV'] = gtv
    
        mask_seg = seg==target

        target_umap = umap[target]
        mask_umap = target_umap>th

        
        roi = mask_seg

        if not only_seg_roi:
            if np.sum(mask_seg) != 0:
                union_roi = (mask_seg + mask_umap)
                union_roi[union_roi>0] = 1
                roi = union_roi
        

        seg_entropy = target_umap * roi
        seg_entropy = seg_entropy[seg_entropy> 0] 



        if np.sum(mask_seg) == 0:
            # no segmentation made for this target (mostly GTV-N)
            entropy_results["Total Entropy"] = np.nan 
            entropy_results["Mean Entropy"] = np.nan 
            entropy_results["Entropy STD"] = np.nan 
            entropy_results["Entropy Volume"] = np.nan 
            entropy_results["Entropy Coefficient of Variation"] = np.nan 
            entropy_results["Logarithm Entropy Coefficient of Variation"] = np.nan 
            # entropy_results["Uncertainty-Segmentation Hausdorff distance"] = np.nan 
            # entropy_results["Uncertainty-Segmentation Mean Surface Distance"] = np.nan 

        else:
            # eros_seg  = binary_erosion(mask_seg, iterations=2).astype(int)
            # seg_line = mask_seg.astype(int) - eros_seg
            # e_hd  = hd(mask_umap, seg_line)
            # e_asd = asd(mask_umap, seg_line)
            entropy_mean = np.mean(seg_entropy) 
            entropy_std = np.std(seg_entropy) 
            entropy_results["Total Entropy"] = np.sum(seg_entropy)
            entropy_results["Mean Entropy"] = entropy_mean
            entropy_results["Entropy STD"] = entropy_std
            entropy_results["Entropy Volume"] = np.sum(roi) 
            entropy_results["Entropy Coefficient of Variation"] = entropy_std / (entropy_mean  + 1e-6) # + 1e-6 to avoid divide by 0
            entropy_results["Logarithm Entropy Coefficient of Variation"] = np.log(entropy_std / (entropy_mean  + 1e-6)) 
            # entropy_results["Uncertainty-Segmentation Hausdorff distance"] = e_hd
            # entropy_results["Uncertainty-Segmentation Mean Surface Distance"] = e_asd

        resutl_list.append(entropy_results)

    return resutl_list

## directorys for internal  probability maps 

DROP2_PMAPS="/data/jintao/nnUNet/nnUNet_results/nnUNet/3d_fullres/Task901_AUH/nnUNetTrainerV2_dropout__nnUNetPlansv2.1/prob_maps/Task901_AUH/imagesTs"
NNUNET_PMAPS="/data/jintao/nnUNet/nnUNet_results/nnUNet/3d_fullres/Task901_AUH/nnUNetTrainerV2__nnUNetPlansv2.1/prob_maps/Task901_AUH/imagesTs"
PH_PMAPS="/data/jintao/nnUNet/nnUNet_results/nnUNet/3d_fullres/Task901_AUH/nnUNetTrainerV2_PhiSeg__nnUNetPlansv2.1/prob_maps/Task901_AUH/imagesTs"
PH_GAMM_PMAP="/data/jintao/nnUNet/nnUNet_results/nnUNet/3d_fullres/Task901_AUH/nnUNetTrainerV2_PhiSeg_gamma__nnUNetPlansv2.1/prob_maps/Task901_AUH/imagesTs"


groups = {'Baseline': join(NNUNET_PMAPS,'f0_tta')
          ,'No TTA': join(NNUNET_PMAPS,'f0')
          ,'MC Dropout': join(DROP2_PMAPS,'f0_mc10_tta')
          ,'Ensemble': join(NNUNET_PMAPS,'f01234_tta')
          ,'Snapshots':join(DROP2_PMAPS,'f0_tta_snap')
          ,"Complex":join(DROP2_PMAPS,'f01234_mc10_tta_snap')
          ,'PhiSeg': join(PH_PMAPS,'f01234_tta')
          }


In [ ]:

Task = 'Complex' #Complex #PhiSeg #External #Baseline #Snapshots
    ## complex
seg_folder = groups[Task]
umap_folder = seg_folder.replace("prob_maps","umaps")

## external
# seg_folder = '/data/jintao/nnUNet/nnUNet_results/nnUNet/3d_fullres/Task901_AUH/nnUNetTrainerV2_dropout__nnUNetPlansv2.1/prob_maps/Task711_NKI_origin/imagesAll_ct_correct/f0_mc10_tta' #f01234_mc10_tta'
# umap_folder =  '/data/jintao/nnUNet/nnUNet_results/nnUNet/3d_fullres/Task901_AUH/nnUNetTrainerV2_dropout__nnUNetPlansv2.1/umaps/Task711_NKI_origin/imagesAll_ct_correct/f0_mc10_tta' #f01234_mc10_tta'

seg_files_in = subfiles(seg_folder, suffix=".nii.gz", join=False) 
umaps = [umap_folder] * len(seg_files_in)
segs = [seg_folder] * len(seg_files_in)

p = Pool(128)
entropy_results=  p.map(calcualte_target_entropy, zip(umaps, segs, seg_files_in))
p.close()
p.join()
#=========================================#
entropy_t = []
entropy_n = []
for result in entropy_results:
    entropy_t.append(result[0])
    entropy_n.append(result[1])
entropy_dft = pd.DataFrame(entropy_t)
entropy_dfn = pd.DataFrame(entropy_n)

entropy_df_all = pd.concat([entropy_dft, entropy_dfn], axis=0)
entropy_df_all = entropy_df_all.reset_index(drop=False)

complex_all_data = all_seg_internal[all_seg_internal["Ucertainty estimation methods"]==Task]
complex_all_data_entropy = pd.merge(complex_all_data, entropy_df_all, how="inner", on=["PatientID", "GTV"])
